In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

--2020-05-29 13:18:47--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M   187MB/s    in 1.2s    

2020-05-29 13:18:49 (187 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import random

In [0]:
sentences=[]
labels=[]
with open('/tmp/training_cleaned.csv','r') as csvfile:
  csvreader=csv.reader(csvfile)
  for i in csvreader:
    sentences.append(i[5])
    labels.append(int(i[0]))
    

In [0]:
len(sentences),len(labels)

(1600000, 1600000)

In [0]:
## Splitting of data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_data,test_data,train_label,test_label=train_test_split(sentences,labels,test_size=0.2)

In [0]:
embed_dim=100
padding='post'
truncating='post'
oov_token='<OOV>'
max_length=16
training_length=160000

In [0]:
train_data=sentences[:training_length]
test_data=sentences[training_length:]
train_label=labels[:training_length]
test_label=labels[training_length:]

In [0]:
#Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
token=Tokenizer(oov_token=oov_token)
token.fit_on_texts(sentences)
word_index=token.word_index
vocab_size=len(word_index)

In [0]:
train_seq=token.texts_to_sequences(train_data)
test_seq=token.texts_to_sequences(test_data)
train_label=np.array(train_label)
test_label=np.array(test_label)

In [0]:
train_label.shape

(160000,)

In [0]:
## Pad sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
train_pad_seq=pad_sequences(train_seq,maxlen=16,padding=padding,truncating=truncating)
test_pad_seq=pad_sequences(test_seq,maxlen=max_length,padding=padding,truncating=truncating)

In [0]:
vocab_size

690961

In [0]:
### Get the Embedding weights from glove data set from Standford.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

--2020-05-29 13:28:53--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   171MB/s    in 1.9s    

2020-05-29 13:28:55 (171 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [0]:

with open('/tmp/glove.6B.100d.txt','r',encoding='utf-8') as glovefile:

  for lines in glovefile:

    print(lines)
    print(len(lines))
    break

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

863


In [0]:
embed_matrix={}
with open('/tmp/glove.6B.100d.txt','r',encoding='utf-8') as glovefile:
  
  for lines in glovefile:
    glovelist=lines.split()
    word=glovelist[0]
    vector=np.array(glovelist[1:],dtype=np.float32)
    embed_matrix[word]=vector



In [0]:
embed_matrix['the'].shape

(100,)

In [0]:
## Map to the word_index dict.
## We are addingmbecause the input dim should be len(word_index) + 1 but not the len(word_index).
embed_vector=np.zeros((vocab_size+1,100),dtype=np.float32)
for word,index in word_index.items():
  embed_word= embed_matrix.get(word)

  if embed_word is not None:
    embed_vector[index]=embed_matrix[word]



NameError: ignored

In [0]:
embed_vector.shape

(690962, 100)

In [0]:
embed_vector[4]

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [0]:
### Lets create model
from tensorflow.keras.layers import Embedding,Dense,Conv1D,MaxPooling1D,LSTM

In [0]:
embed_vector.shape

(690962, 100)

In [0]:
model=keras.Sequential()
## WE will pass the embed vector and set the training mode False as we already have weights.
model.add(Embedding(input_dim=vocab_size+1,output_dim=100,input_length=max_length,weights=[embed_vector],trainable=False))
## We have many more dimensions so lets reduce the words without changing the meaning .This can be done as well using Convolution
### WE are coverting 100 dim to 64 and reducing the size and then passing to LSTM
model.add(Conv1D(filters=64,kernel_size=5,activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(64))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 16, 100)           69096200  
_________________________________________________________________
conv1d (Conv1D)              (None, 12, 64)            32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3, 64)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 69,161,353
Trainable params: 65,153
Non-trainable params: 69,096,200
_________________________________________________________________


In [0]:
model.fit(train_pad_seq,train_label,epochs=15,validation_data=(test_pad_seq,test_label))

Epoch 1/15
5000/5000 [==============================] - 161s 32ms/step - loss: 0.6935 - accuracy: 0.9998 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 2/15
5000/5000 [==============================] - 157s 31ms/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 3/15
5000/5000 [==============================] - 154s 31ms/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 4/15
5000/5000 [==============================] - 156s 31ms/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 5/15
5000/5000 [==============================] - 154s 31ms/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 6/15
5000/5000 [==============================] - 156s 31ms/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6933 - val_accuracy: 0.4444
Epoch 7/15
5000/5000 [==============================] - 155s 31ms/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0

In [0]:
[5,4)

SyntaxError: ignored